In [1]:
!pip install great-expectations
!pip install pyspark
import great_expectations as gx
from pyspark.sql import SparkSession
from great_expectations.expectations import ExpectColumnMinToBeBetween

In [2]:
file_path = "yellow_tripdata_2024-01.parquet"
spark = SparkSession.builder.appName("ParquetReader").getOrCreate()
df = spark.read.parquet(file_path)
#print(list(df.columns))
#print(df.show(5))
context = gx.get_context()
data_source = context.data_sources.add_spark("data_source")
data_asset = data_source.add_dataframe_asset("data_asset")
batch_definition = data_asset.add_batch_definition_whole_dataframe("batch_definition")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/06 16:49:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
def test_ExpectColumnMinToBeBetween():
    print(df.limit(5).show())
    expectation_instance = ExpectColumnMinToBeBetween(
        column="extra",
        min_value=0,
        max_value=3,
        strict_max=False
        )
    batch = batch_definition.get_batch(batch_parameters={"dataframe": df.limit(5)})
    validation_results = batch.validate(expectation_instance)
    print(validation_results)

test_ExpectColumnMinToBeBetween()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-01-01 00:57:55|  2024-01-01 01:17:43|              1|         1.72|         1|                 N|         186|          79|           2|       17.7|  1.0|    0.5|       0.

Calculating Metrics: 100%|██████████| 6/6 [00:01<00:00,  4.84it/s]

{
  "success": true,
  "expectation_config": {
    "type": "expect_column_min_to_be_between",
    "kwargs": {
      "batch_id": "data_source-data_asset",
      "column": "extra",
      "min_value": 0.0,
      "max_value": 3.0
    },
    "meta": {}
  },
  "result": {
    "observed_value": 1.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}
